# Actividad 2: PRÁCTICA DE CLASIFICACIÓN DE TEXTOS
## Borja Lacalle Álvarez

In [1]:
#importar librerias para el desarrollo de la actividad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import nltk

In [2]:
# leer dataset spam.csv
spam = pd.read_csv('spam.csv')
spam.head(10)



,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
# ver porcentaje de spam y no spam
spam['label'].value_counts(normalize=True)


label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

In [4]:
#normalizar el texto quitando signos de puntuacion y stopwords, entre otras cosas
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

wpt = WordPunctTokenizer()
stop_words = set(stopwords.words('english'))

def normalize_document(doc):
    doc = doc.replace("!", "").replace("¡", "").replace(",", "").replace(".", "").replace(";", "")
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [5]:
norm_spam=[]

for document in spam['text']:
    norm_spam.append(normalize_document(document))

#ver solo los primeros 20 documentos normalizados
norm_spam[:20]


['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 "free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question ( std txt rate ) & c ' apply 08452810075over18 '",
 'u dun say early hor u c already say',
 "nah ' think goes usf lives around though",
 "freemsg hey darling ' 3 week ' word back ' like fun still ? tb ok xxx std chgs send å £ 150 rcv",
 'even brother like speak treat like aids patent',
 "per request ' melle melle ( oru minnaminunginte nurungu vettam )' set callertune callers press * 9 copy friends callertune",
 'winner valued network customer selected receivea å £ 900 prize reward claim call 09061701461 claim code kl341 valid 12 hours',
 'mobile 11 months ? u r entitled update latest colour mobiles camera free call mobile update co free 08002986030',
 "' gonna home soon ' want talk stuff anymore tonight k ? ' cried enough today",
 'six chances win cash 100 20000 pounds txt > c

In [6]:
#convertir norm_spam en una matriz tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(norm_spam)

tfidf_array = tfidf.toarray()
tfidf_array.shape


(5572, 9042)

In [7]:
#dividir el dataset en train (80%) y test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf, spam['label'], test_size=0.2, random_state=42)


#### Clasificador bayesiano ingenuo (Naive Bayes)

In [8]:
# entenar modelo bayesiano ingenuo
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)


MultinomialNB()

In [9]:
#predecir con el modelo entrenado
y_pred_nb = nb.predict(X_test)
y_pred_nb

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'spam'], dtype='<U4')

In [10]:
#evaluar el modelo con la matriz de confusion
from sklearn.metrics import confusion_matrix

nb_conf = confusion_matrix(y_test, y_pred_nb)
nb_conf


array([[965,   0],
       [ 37, 113]])

La estructura típica de una matriz de confusión binaria es la siguiente:

[[Verdaderos Negativos (TN), Falsos Positivos (FP)]  
 [Falsos Negativos (FN), Verdaderos Positivos (TP)]]

Por lo tanto, estos valores tienen el siguiente significado:

TN (True Negatives): La cantidad de mensajes clasificados correctamente como "ham" (no spam).  
FP (False Positives): La cantidad de mensajes que fueron clasificados incorrectamente como "spam" cuando en realidad son "ham".  
FN (False Negatives): La cantidad de mensajes que fueron clasificados incorrectamente como "ham" cuando en realidad son "spam".  
TP (True Positives): La cantidad de mensajes clasificados correctamente como "spam".



In [11]:
# evaluar el modelo con accuracy 
from sklearn.metrics import accuracy_score

nb_acc = accuracy_score(y_test, y_pred_nb)
nb_acc


0.9668161434977578

El modelo acierta el 96.68% de las predicciones en el conjunto de evaluación. Es una medida de qué tan bien el modelo está clasificando correctamente los mensajes como spam o no spam.

#### Máquina SVM (SUPPORT VECTOR MACHINE)

In [12]:
# entrenar el modelo de svm
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [13]:
# predecir con el modelo entrenado
y_pred_svm = svm.predict(X_test)
y_pred_svm

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [14]:
# evaluar el modelo con la matriz de confusion
svm_conf = confusion_matrix(y_test, y_pred_svm)
svm_conf

array([[963,   2],
       [ 29, 121]])

In [15]:
# evaluar el modelo con accuracy
svm_acc = accuracy_score(y_test, y_pred_svm)
svm_acc

0.9721973094170404

#### Árboles de Decisión


In [16]:
# entrenar el modelo de arbol de decision
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)


DecisionTreeClassifier()

In [17]:
# predecir con el modelo entrenado
y_pred_dt = dt.predict(X_test)
y_pred_dt

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'spam'], dtype=object)

In [18]:
# evaluar el modelo con la matriz de confusion
dt_conf = confusion_matrix(y_test, y_pred_dt)
dt_conf

array([[951,  14],
       [ 20, 130]])

In [19]:
# evaluar el modelo con accuracy
dt_acc = accuracy_score(y_test, y_pred_dt)
dt_acc

0.9695067264573991

#### Comparación

In [20]:
# Store the evaluation metrics in a dictionary for easy comparison
model_metrics = {
    "Naive Bayes": {
        "Accuracy": nb_acc,
        "Confusion Matrix": nb_conf
    },
    "SVM": {
        "Accuracy": svm_acc,
        "Confusion Matrix": svm_conf
    },
    "Decision Tree": {
        "Accuracy": dt_acc,
        "Confusion Matrix": dt_conf
    }
}

# Function to print the comparison
def print_model_comparison_percentage(metrics):
    for model, data in metrics.items():
        accuracy_percentage = data['Accuracy'] * 100
        print(f"Model: {model}")
        print(f"  Accuracy: {accuracy_percentage:.2f}%")
        print(f"  Confusion Matrix:\n{data['Confusion Matrix']}\n")

# Print the comparison with updated format
print_model_comparison_percentage(model_metrics)

Model: Naive Bayes
  Accuracy: 96.68%
  Confusion Matrix:
[[965   0]
 [ 37 113]]

Model: SVM
  Accuracy: 97.22%
  Confusion Matrix:
[[963   2]
 [ 29 121]]

Model: Decision Tree
  Accuracy: 96.95%
  Confusion Matrix:
[[951  14]
 [ 20 130]]



#### ¿Tiene influencia en el resultado final el número máximo de features a utilizar?

##### 1000 features 

In [21]:
# Reinitialize the TF-IDF Vectorizer with max_features set to 1000
tfidf_vectorizer_1000 = TfidfVectorizer(max_features=1000)

# Fit and transform the processed text
tfidf_1000 = tfidf_vectorizer_1000.fit_transform(norm_spam)

# Split the data into training and testing sets
X_train_1000, X_test_1000, y_train_1000, y_test_1000 = train_test_split(tfidf_1000, spam['label'], test_size=0.2, random_state=42)

# Reinitialize and train the Naive Bayes classifier
nb_classifier_1000 = MultinomialNB()
nb_classifier_1000.fit(X_train_1000, y_train_1000)

# Reinitialize and train the SVM classifier
svm_classifier_1000 = SVC()
svm_classifier_1000.fit(X_train_1000, y_train_1000)

# Reinitialize and train the Decision Tree classifier
dt_classifier_1000 = DecisionTreeClassifier()
dt_classifier_1000.fit(X_train_1000, y_train_1000)

# Predict and evaluate the models
y_pred_nb_1000 = nb_classifier_1000.predict(X_test_1000)
y_pred_svm_1000 = svm_classifier_1000.predict(X_test_1000)
y_pred_dt_1000 = dt_classifier_1000.predict(X_test_1000)

# Store the evaluation metrics for the new models
model_metrics_1000 = {
    "Naive Bayes (1000 features)": {
        "Accuracy": nb_classifier_1000.score(X_test_1000, y_test_1000),
        "Confusion Matrix": confusion_matrix(y_test_1000, y_pred_nb_1000)
    },
    "SVM (1000 features)": {
        "Accuracy": svm_classifier_1000.score(X_test_1000, y_test_1000),
        "Confusion Matrix": confusion_matrix(y_test_1000, y_pred_svm_1000)
    },
    "Decision Tree (1000 features)": {
        "Accuracy": dt_classifier_1000.score(X_test_1000, y_test_1000),
        "Confusion Matrix": confusion_matrix(y_test_1000, y_pred_dt_1000)
    }
}

# Print the comparison with updated format
print_model_comparison_percentage(model_metrics_1000)

# print the shape of the new TF-IDF Vectorizer
print(tfidf_1000.toarray().shape)


Model: Naive Bayes (1000 features)
  Accuracy: 97.94%
  Confusion Matrix:
[[962   3]
 [ 20 130]]

Model: SVM (1000 features)
  Accuracy: 97.94%
  Confusion Matrix:
[[963   2]
 [ 21 129]]

Model: Decision Tree (1000 features)
  Accuracy: 96.05%
  Confusion Matrix:
[[946  19]
 [ 25 125]]

(5572, 1000)


##### 5000 features

In [22]:
# Reinitialize the TF-IDF Vectorizer with max_features set to 5000
tfidf_vectorizer_5000 = TfidfVectorizer(max_features=5000)

# Fit and transform the processed text
tfidf_5000 = tfidf_vectorizer_5000.fit_transform(norm_spam)

# Split the data into training and testing sets
X_train_5000, X_test_5000, y_train_5000, y_test_5000 = train_test_split(tfidf_5000, spam['label'], test_size=0.2, random_state=42)

# Reinitialize and train the Naive Bayes classifier
nb_classifier_5000 = MultinomialNB()
nb_classifier_5000.fit(X_train_5000, y_train_5000)

# Reinitialize and train the SVM classifier
svm_classifier_5000 = SVC()
svm_classifier_5000.fit(X_train_5000, y_train_5000)

# Reinitialize and train the Decision Tree classifier
dt_classifier_5000 = DecisionTreeClassifier()
dt_classifier_5000.fit(X_train_5000, y_train_5000)

# Predict and evaluate the models
y_pred_nb_5000 = nb_classifier_5000.predict(X_test_5000)
y_pred_svm_5000 = svm_classifier_5000.predict(X_test_5000)
y_pred_dt_5000 = dt_classifier_5000.predict(X_test_5000)

# Store the evaluation metrics for the new models
model_metrics_5000 = {
    "Naive Bayes (5000 features)": {
        "Accuracy": nb_classifier_5000.score(X_test_5000, y_test_5000),
        "Confusion Matrix": confusion_matrix(y_test_5000, y_pred_nb_5000)
    },
    "SVM (5000 features)": {
        "Accuracy": svm_classifier_5000.score(X_test_5000, y_test_5000),
        "Confusion Matrix": confusion_matrix(y_test_5000, y_pred_svm_5000)
    },
    "Decision Tree (5000 features)": {
        "Accuracy": dt_classifier_5000.score(X_test_5000, y_test_5000),
        "Confusion Matrix": confusion_matrix(y_test_5000, y_pred_dt_5000)
    }
}

# Print the comparison with updated format
print_model_comparison_percentage(model_metrics_5000)


Model: Naive Bayes (5000 features)
  Accuracy: 97.49%
  Confusion Matrix:
[[965   0]
 [ 28 122]]

Model: SVM (5000 features)
  Accuracy: 97.49%
  Confusion Matrix:
[[963   2]
 [ 26 124]]

Model: Decision Tree (5000 features)
  Accuracy: 96.32%
  Confusion Matrix:
[[947  18]
 [ 23 127]]



##### Comparación accuracy de los modelos sin limite, con 1000 y 5000 features

In [23]:
# Valores de precisión previamente calculados y discutidos
accuracy_values = {
    "Naive Bayes": {"Sin Límite": 96.68, "1000 Features": 97.94, "5000 Features": 97.49},
    "SVM": {"Sin Límite": 97.22, "1000 Features": 97.94, "5000 Features": 97.49},
    "Decision Tree": {"Sin Límite": 96.95, "1000 Features": 96.41, "5000 Features": 96.41}
}

# Crear DataFrame para la comparación
accuracy_df = pd.DataFrame(accuracy_values).T

# Imprimir la tabla de comparación
accuracy_df



,Sin Límite,1000 Features,5000 Features
Naive Bayes,96.68,97.94,97.49
SVM,97.22,97.94,97.49
Decision Tree,96.95,96.41,96.41


#### ¿Modifica el resultado si no se eliminan las stop words?
Dado que la vectorización con 1000 características mostró una mejora general en el rendimiento para los modelos de Naive Bayes y SVM, y un rendimiento estable para Decision Tree, me centraré en este escenario. Además, 1000 características representan un buen equilibrio entre la retención de información importante y la reducción de la dimensionalidad, lo cual es crucial para entender el impacto de las stopwords.

In [24]:
#normalizar el texto quitando signos de puntuacion y dejando las stopwords
from nltk.tokenize import WordPunctTokenizer

wpt = WordPunctTokenizer()

def normalize_document_stop(doc):
    doc = doc.replace("!", "").replace("¡", "").replace(",", "").replace(".", "").replace(";", "").replace("'", "").replace('"', "").replace("?", "").replace("¿", "").replace(":", "")
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # re-create document
    doc = ' '.join(tokens)
    return doc

In [25]:
norm_spam_stop=[]

for document in spam['text']:
    norm_spam_stop.append(normalize_document_stop(document))

#ver solo los primeros 20 documentos normalizados
norm_spam_stop[:20]

['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat',
 'ok lar joking wif u oni',
 'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question ( std txt rate ) t & cs apply 08452810075over18s',
 'u dun say so early hor u c already then say',
 'nah i dont think he goes to usf he lives around here though',
 'freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å £ 150 to rcv',
 'even my brother is not like to speak with me they treat me like aids patent',
 'as per your request melle melle ( oru minnaminunginte nurungu vettam ) has been set as your callertune for all callers press * 9 to copy your friends callertune',
 'winner as a valued network customer you have been selected to receivea å £ 900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only',
 'had your mobile 11 months or more u r

In [26]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_stop = tfidf_vectorizer.fit_transform(norm_spam_stop)

tfidf_array_stop = tfidf_stop.toarray()
tfidf_array_stop.shape


(5572, 1000)

In [27]:
#dividir el dataset en train (80%) y test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_stop, spam['label'], test_size=0.2, random_state=42)

#### skjdbv

In [28]:
# entenar modelo bayesiano ingenuo, svm y arbol de decision
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

nb = MultinomialNB()
svm = SVC()
dt = DecisionTreeClassifier()

nb.fit(X_train, y_train)
svm.fit(X_train, y_train)
dt.fit(X_train, y_train)

#predecir con el modelo entrenado
y_pred_nb = nb.predict(X_test)
y_pred_svm = svm.predict(X_test)
y_pred_dt = dt.predict(X_test)

#evaluar el modelo con accuracy
nb_acc_stop = accuracy_score(y_test, y_pred_nb)
svm_acc_stop = accuracy_score(y_test, y_pred_svm)
dt_acc_stop = accuracy_score(y_test, y_pred_dt)

#evaluar el modelo con la matriz de confusion
nb_conf_stop = confusion_matrix(y_test, y_pred_nb)
svm_conf_stop = confusion_matrix(y_test, y_pred_svm)
dt_conf_stop = confusion_matrix(y_test, y_pred_dt)

# ver los resultados de accuracy con tres decimales y en porcentaje y la matriz de confusion
print(f"Naive Bayes Accuracy: {nb_acc*100:.2f}%")
print(f"Naive Bayes Confusion Matrix:\n{nb_conf}\n")
print(f"SVM Accuracy: {svm_acc*100:.2f}%")
print(f"SVM Confusion Matrix:\n{svm_conf}\n")
print(f"Decision Tree Accuracy: {dt_acc*100:.2f}%")
print(f"Decision Tree Confusion Matrix:\n{dt_conf}\n")




Naive Bayes Accuracy: 96.68%
Naive Bayes Confusion Matrix:
[[965   0]
 [ 37 113]]

SVM Accuracy: 97.22%
SVM Confusion Matrix:
[[963   2]
 [ 29 121]]

Decision Tree Accuracy: 96.95%
Decision Tree Confusion Matrix:
[[951  14]
 [ 20 130]]



el resultado es el mismo que entrenando los modelos quitando las stopwords pero sin limite de features. Es decir que la mejor combinación según el accuracy de los modelos sería quitar las stopwords y poner un limite de 1000 features

#### **Informe de Análisis de Clasificacion de Spam**

Este informe presenta los resultados de un proyecto de clasificación de correos electrónicos en categorías de "spam" y "no spam" (ham). Se utilizó un conjunto de datos de correos electrónicos, y se aplicaron técnicas de Procesamiento de Lenguaje Natural (NLP) para transformar el texto en datos numéricos como es la matriz tfidf que permite cuantificar la importancia de palabras (o términos) en documentos.   
El conjunto de datos consta de 5,572 correos electrónicos, etiquetados como 'spam' o 'ham'. Cada correo fue preprocesado mediante técnicas estándar de NLP, incluyendo la conversión a minúsculas, eliminación de caracteres especiales y puntuación, y tokenización para quitar despues las stopwords (palabras que a priori no añaden significado a los textos por su caracter común).
  
Se probaron tres modelos de aprendizaje automático: **Naive Bayes, Máquinas de Soporte Vectorial (SVM) y Árboles de Decisión**, con diferentes configuraciones de características para optimizar su rendimiento.   
Se experimentó con tres configuraciones de vectorización TF-IDF:

    - Sin límite de características
    - Limitando a 1,000 características
    - Limitando a 5,000 características
Además, se consideró el impacto de la inclusión y exclusión de stopwords en la vectorización.

##### </u>**Resultados**</u>
Los modelos mostraron variaciones en su rendimiento según la configuración:

**Naive Bayes**: Mejoró significativamente con la reducción a 1,000 características. La precisión pasó del 96.68% sin límite a 97.94% con 1,000 características. Con 5,000 características, la precisión fue ligeramente menor (97.49%) pero aun asi mejor que sin límite.

**SVM**: Este modelo también mostró una mejora con 1,000 características, alcanzando una precisión del 97.94%. Con 5,000 características, la precisión fue un poco menor pero aún mejor que sin límite (97.49% frente a 97.22%).

**Decision Tree**: Mostró un rendimiento más estable a través de diferentes configuraciones, aunque ligeramente mejor sin límite (96.95%).

***Descubrimientos Clave***  

La reducción de características mejora el rendimiento en Naive Bayes y SVM, probablemente al reducir el ruido y el overfitting. 

Los árboles de decisión son menos sensibles a la reducción de características, lo que sugiere una mayor robustez a diferentes representaciones de características. Es decir que no se ven tan afectados por la presencia o ausencia de ciertas características  

La inclusión de stopwords podría ser relevante para evaluar, ya que su impacto puede variar según el modelo y el contexto del análisis. dicho esto, en las pruebas que he hecho, el unico modelo que se comportaba mejor con stopwords es el de decision trees, probablemente por lo mencionado en el punto anterior.

Para futuras implementaciones, se recomienda utilizar el modelo SVM con 1,000 características y quitando stopwords para un equilibrio óptimo entre precisión y complejidad del modelo. Ademas, cabe considerar la experimentación con técnicas de ajuste de hiperparámetros y modelos más avanzados para posibles mejoras

En conclusión, el proyecto demostró la viabilidad de clasificar eficazmente correos electrónicos como 'spam' o 'ham' utilizando técnicas de NLP y aprendizaje automático. Los modelos SVM y Naive Bayes mostraron un rendimiento particularmente prometedor, con mejoras significativas en la precisión al ajustar el número de características en la vectorización TF-IDF.
